In [150]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim


In [151]:
train_transform=transforms.Compose([
    transforms.RandomCrop(30),
    transforms.ColorJitter(0.2,0.2),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [152]:
test_transform=transforms.Compose([
    transforms.Resize((30,30)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [153]:
t_dataset=datasets.CIFAR10(root='./data',train=True,transform=train_transform,download=True)
te_dataset=datasets.CIFAR10(root='./data',train=False,transform=test_transform,download=True)

In [154]:
train_loader=DataLoader(t_dataset,128,True)
test_loader=DataLoader(te_dataset,128,True)

In [155]:
print(len(train_loader))

391


In [156]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [157]:
device

'cuda'

In [158]:
images,labels=next(iter(train_loader))

In [159]:
images.shape

torch.Size([128, 3, 30, 30])

In [160]:
model = nn.Sequential(
    nn.Conv2d(3,32,3,padding=1),
    nn.ReLU(),
    nn.Conv2d(32,32,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),       # 30 -> 15

    nn.Conv2d(32,64,3,padding=1),
    nn.ReLU(),
    nn.Conv2d(64,64,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),       # 15 -> 7

    nn.Conv2d(64,128,3,padding=1),
    nn.ReLU(),
    nn.Conv2d(128,128,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2),       # 7 -> 3

    nn.Flatten(),
    nn.Linear(128*3*3, 256), # 128*3*3 = 1152 ✅
    nn.ReLU(),
    nn.Linear(256,10)
)


In [161]:
output=model(images)

In [162]:
model=model.to(device)


In [163]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
for epoch in range(50):
    model.train()
    for images,labels in train_loader:
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        output=model(images)
        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct,total=0,0
    with torch.no_grad():
        for images,labels in test_loader:
            images,labels=images.to(device),labels.to(device)
            output=model(images)
            _,predicted=torch.max(output,1)
            total+=labels.size(0)
            correct+=(predicted==labels).sum().item()
    accuracy=correct/total

    print(f"For Epoch {epoch+1}")
    print(f"        Val Loss: {accuracy}")

For Epoch 1
        Val Loss: 0.4247
For Epoch 2
        Val Loss: 0.5144
For Epoch 3
        Val Loss: 0.5331
For Epoch 4
        Val Loss: 0.5653
For Epoch 5
        Val Loss: 0.5743
For Epoch 6
        Val Loss: 0.6302
For Epoch 7
        Val Loss: 0.6288
For Epoch 8
        Val Loss: 0.6364
For Epoch 9
        Val Loss: 0.6687
For Epoch 10
        Val Loss: 0.6657
For Epoch 11
        Val Loss: 0.702
For Epoch 12
        Val Loss: 0.6957
For Epoch 13
        Val Loss: 0.7136
For Epoch 14
        Val Loss: 0.7033
For Epoch 15
        Val Loss: 0.7128
For Epoch 16
        Val Loss: 0.7021
For Epoch 17
        Val Loss: 0.7272
For Epoch 18
        Val Loss: 0.749
For Epoch 19
        Val Loss: 0.744
For Epoch 20
        Val Loss: 0.7321
For Epoch 21
        Val Loss: 0.7355
For Epoch 22
        Val Loss: 0.7574
For Epoch 23
        Val Loss: 0.7397
For Epoch 24
        Val Loss: 0.7407
For Epoch 25
        Val Loss: 0.7603
For Epoch 26
        Val Loss: 0.7569
For Epoch 27
        Val

In [ ]:
torch.save(model.state_dict(), 'cifar10.pth')
